In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import pysolar.solar as solar
import sqlite3
from metar import Metar
from metar_taf_parser.parser.parser import TAFParser
import metar_tools
import taf_tools
import misc_tools

efhk_lat = 60.3183
efhk_lon = 24.9497

ilzq_lat = 60.6277
ilzq_lon = 25.1934

In [2]:
cols = ['ttime', 'content']
efhk_metar = pd.read_csv('sample-data/EFHK_2021_METAR.csv', sep=';', usecols=cols)
efhk_metar['time'] = pd.to_datetime(efhk_metar['ttime'])
efhk_metar.set_index('time', inplace=True)

efhk_metar.index

DatetimeIndex(['2021-01-01 00:20:00', '2021-01-01 00:50:00',
               '2021-01-01 01:20:00', '2021-01-01 01:50:00',
               '2021-01-01 02:20:00', '2021-01-01 02:50:00',
               '2021-01-01 03:20:00', '2021-01-01 03:50:00',
               '2021-01-01 04:20:00', '2021-01-01 04:50:00',
               ...
               '2021-12-03 05:50:00', '2021-12-03 06:20:00',
               '2021-12-03 06:50:00', '2021-12-03 07:20:00',
               '2021-12-03 07:50:00', '2021-12-03 08:20:00',
               '2021-12-03 08:50:00', '2021-12-03 09:20:00',
               '2021-12-03 09:50:00', '2021-12-03 10:20:00'],
              dtype='datetime64[ns]', name='time', length=16140, freq=None)

In [3]:
cols = ['ttime', 'content']
efhk_taf = pd.read_csv('sample-data/EFHK_2021_TAF.csv', sep=';', usecols=cols)
efhk_taf['time'] = pd.to_datetime(efhk_taf['ttime'])
efhk_taf.set_index('time', inplace=True)
efhk_taf = efhk_taf.asfreq('10min', method='ffill')
efhk_taf['content'] = efhk_taf['content'].ffill()
efhk_taf.iloc[-35:,:]

,ttime,content
time,,
2021-12-08 08:20:00,2021-12-08 08:00:00,EFHK 080659Z 0806/0906 32004KT 9999 FEW002 PRO...
2021-12-08 08:30:00,2021-12-08 08:00:00,EFHK 080659Z 0806/0906 32004KT 9999 FEW002 PRO...
2021-12-08 08:40:00,2021-12-08 08:00:00,EFHK 080659Z 0806/0906 32004KT 9999 FEW002 PRO...
2021-12-08 08:50:00,2021-12-08 08:00:00,EFHK 080659Z 0806/0906 32004KT 9999 FEW002 PRO...
2021-12-08 09:00:00,2021-12-08 09:00:00,EFHK 080838Z 0809/0909 32003KT 9999 FEW002 TEM...
2021-12-08 09:10:00,2021-12-08 09:00:00,EFHK 080838Z 0809/0909 32003KT 9999 FEW002 TEM...
2021-12-08 09:20:00,2021-12-08 09:00:00,EFHK 080838Z 0809/0909 32003KT 9999 FEW002 TEM...
2021-12-08 09:30:00,2021-12-08 09:00:00,EFHK 080838Z 0809/0909 32003KT 9999 FEW002 TEM...
2021-12-08 09:40:00,2021-12-08 09:00:00,EFHK 080838Z 0809/0909 32003KT 9999 FEW002 TEM...


In [4]:
ilzq_metar = pd.read_csv('sample-data/ILZQ_2021_METAR.dat', sep=';', usecols=cols)
ilzq_metar['time'] = pd.to_datetime(ilzq_metar['ttime'])
ilzq_metar.set_index('time', inplace=True)
ilzq_metar

,ttime,content
time,,
2021-01-01 00:00:00,2021-01-01 00:00:00,ILZQ 010000Z AUTO 23003KT 2300 BR ////// 00/00...
2021-01-01 00:10:00,2021-01-01 00:10:00,ILZQ 010010Z AUTO 23003KT 2100 BR ////// M00/M...
2021-01-01 00:20:00,2021-01-01 00:20:00,ILZQ 010020Z AUTO 23003KT 1800 BR ////// M00/M...
2021-01-01 00:40:00,2021-01-01 00:40:00,ILZQ 010040Z AUTO 22003KT 1700 BR ////// M00/M...
2021-01-01 00:50:00,2021-01-01 00:50:00,ILZQ 010050Z AUTO 23003KT 1800 BR ////// M00/M...
...,...,...
2021-12-03 16:40:00,2021-12-03 16:40:00,ILZQ 031640Z AUTO 35005KT 4200 -SN OVC020 M08/...
2021-12-03 16:50:00,2021-12-03 16:50:00,ILZQ 031650Z AUTO 35004KT 4700 -SN OVC021 M08/...
2021-12-03 17:00:00,2021-12-03 17:00:00,ILZQ 031700Z AUTO 36004KT 5000 -SN OVC030 M08/...


In [5]:
metar = Metar.Metar(efhk_metar.iloc[16, 1])
print(metar.string())

station: EFHK
type: routine report, cycle 8 (automatic report)
time: Sat Jan  1 08:20:00 2022
temperature: 0.0 C
dew point: 0.0 C
wind: ESE at 2 knots
visibility: 800 meters
visual range: on runway 04L, greater than 2000 meters; on runway 15, greater than 2000 meters; on runway 22L, 2000 meters; on runway 22R, greater than 2000 meters
pressure: 1011.0 mb
weather: fog
sky: scattered clouds at 400 feet
     broken clouds at 1300 feet
METAR: EFHK 010820Z 11002KT 0800 R04L/P2000N R15/P2000N R22L/2000N R22R/P2000N FG SCT004 BKN013 00/00 Q1011 BECMG 2000 BR=


In [6]:
#metar.runway[0][1].value()
metar.runway[0][1].value()

2000.0

In [7]:
metar.sky

[('SCT', <metar.Datatypes.distance at 0x12074e5fb50>, None),
 ('BKN', <metar.Datatypes.distance at 0x12074e5f880>, None)]

In [8]:
dt = datetime.fromisoformat(efhk_metar.iloc[16, 0] + '+00:00')
print(solar.get_altitude(efhk_lat, efhk_lon, dt))

3.2038041011444016


In [9]:
temp = metar.temp.value()
temp

0.0

In [10]:
vis = int(metar.vis.value())
vis

800

In [11]:
metar_tools.get_rvr(metar)

2000

In [12]:
metar_tools.get_ceil(metar)

1300

In [13]:
metar_tools.get_base(metar, metar_tools.get_ceil(metar))

400

In [14]:
row = 0 #42
ilzq_obs = Metar.Metar(ilzq_metar.iloc[row, 1])
print(ilzq_obs.string())

station: ILZQ
type: routine report (automatic report)
time: Sat Jan  1 00:00:00 2022
temperature: 0.0 C
dew point: 0.0 C
wind: SW at 3 knots
visibility: 2300 meters
pressure: 1009.0 mb
weather: mist; ,  and 
METAR: ILZQ 010000Z AUTO 23003KT 2300 BR ////// 00/00 Q1009=


In [15]:
print(ilzq_obs.sky)

[]


In [16]:
metar_tools.get_ceil(ilzq_obs)

0

In [17]:
metar_tools.get_base(ilzq_obs, metar_tools.get_ceil(ilzq_obs))

0

In [18]:
misc_tools.is_night('2021-01-01 06:10:00', efhk_lat, efhk_lon)

True

In [19]:
t = datetime.strptime('2021-01-01 06:10:00', '%Y-%m-%d %H:%M:%S')

In [20]:
taf = TAFParser().parse('TAF ' + efhk_taf.iloc[11, 1])
taf

In [21]:
taf_tools.validity_during_flight(taf, datetime.strptime('2021-01-07 18:00:00', '%Y-%m-%d %H:%M:%S'), 3)

0

In [22]:
print(taf_tools.get_worstcase_vis(taf, '2021-01-08 15:00:00', 3))

None


In [23]:
def extract_taf(data: np.ndarray):
    time_str = data[0]
    taf_str = data[1]
    forecast = taf_tools.parse(taf_str) #TAFParser().parse('TAF ' + taf_str)
    vis = taf_tools.get_worstcase_vis(forecast, time_str, 3)
    return (time_str, vis, taf_str)

In [24]:
def extract_metar(data: np.ndarray, lat: float, lon: float):
    time_str = data[0]
    metar_str = data[1]
    obs = metar_tools.parse(metar_str) #Metar.Metar(metar_str, strict=False)
    temp = metar_tools.get_temp(obs)
    vis = metar_tools.get_vis(obs)
    rvr = metar_tools.get_rvr(obs)
    ceil = metar_tools.get_ceil(obs)
    base = metar_tools.get_base(obs, ceil)
    night = misc_tools.is_night(time_str, efhk_lat, efhk_lon)
    return (time_str, temp, vis, rvr, ceil, base, night, metar_str)

In [25]:
taf = [extract_taf(d) for d in zip(efhk_taf.ttime, efhk_taf.content)]

In [25]:
a = [extract_metar(d, lat=efhk_lat, lon=efhk_lon) for d in zip(efhk_metar.ttime, efhk_metar.content)]

C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body '400S' while processing 'EFHK 010250Z 32002KT 0900 400S R04L/1300D R15/1200U R22L/0650D R22R/1600U FG BKN002 M12/M13 Q0994 BECMG 2000 BR='
  warnings.warn(message, RuntimeWarning)
C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'BRFG -SHRA BKN002 FEW020CB' while processing 'EFHK 190350Z 36005KT 0800 R04L/P2000N R15/1500D R22L/1700D R22R/1800D -RA FG OVC002 12/12 Q0999 TEMPO 1200 BRFG -SHRA BKN002 FEW020CB='
  warnings.warn(message, RuntimeWarning)
C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'NSW' while processing 'EFHK 261120Z 31004KT 230V350 9999 TS FEW025 FEW030CB 23/17 Q1011 BECMG FEW030 NSW='
  warnings.warn(message, RuntimeWarning)
C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups

In [26]:
b = [extract_metar(d, lat=ilzq_lat, lon=ilzq_lon) for d in zip(ilzq_metar.ttime, ilzq_metar.content)]

In [ ]:
ilzq_df = pd.DataFrame(b, columns=['time', 'temp', 'vis', 'rvr', 'ceil', 'base', 'night', 'metar'])
ilzq_df['time'] = pd.to_datetime(ilzq_df['time'])
ilzq_df.set_index('time', inplace=True)
ilzq_df

,temp,vis,rvr,ceil,base,night,metar
time,,,,,,,
2021-01-01 00:00:00,0,2300,9999,9999,400,True,ILZQ 010000Z AUTO 23003KT 2300 BR ////// 00/00...
2021-01-01 00:10:00,0,2100,9999,9999,400,True,ILZQ 010010Z AUTO 23003KT 2100 BR ////// M00/M...
2021-01-01 00:20:00,0,1800,9999,9999,400,True,ILZQ 010020Z AUTO 23003KT 1800 BR ////// M00/M...
2021-01-01 00:40:00,0,1700,9999,9999,400,True,ILZQ 010040Z AUTO 22003KT 1700 BR ////// M00/M...
2021-01-01 00:50:00,0,1800,9999,9999,400,True,ILZQ 010050Z AUTO 23003KT 1800 BR ////// M00/M...
...,...,...,...,...,...,...,...
2021-12-03 16:40:00,-8,4200,9999,2000,400,True,ILZQ 031640Z AUTO 35005KT 4200 -SN OVC020 M08/...
2021-12-03 16:50:00,-8,4700,9999,2100,400,True,ILZQ 031650Z AUTO 35004KT 4700 -SN OVC021 M08/...
2021-12-03 17:00:00,-8,5000,9999,3000,400,True,ILZQ 031700Z AUTO 36004KT 5000 -SN OVC030 M08/...


In [ ]:
ilzq_df = pd.DataFrame(b, columns=['time', 'temp', 'vis', 'rvr', 'ceil', 'base', 'night', 'metar'])
ilzq_df['time'] = pd.to_datetime(ilzq_df['time'])
ilzq_df.set_index('time', inplace=True)

In [ ]:
efhk_df = pd.DataFrame(a, columns=['time', 'temp', 'vis', 'rvr', 'ceil', 'base', 'night', 'metar'])
efhk_df['time'] = pd.to_datetime(efhk_df['time'])
efhk_df.set_index('time', inplace=True)

In [ ]:
efhk_df = efhk_df.asfreq('10min', method='ffill')

In [ ]:
efhk_ilzq = efhk_df.join(ilzq_df, how='inner', lsuffix='_term', rsuffix='_sect')
efhk_ilzq.drop(columns=['night_term', 'rvr_sect'], inplace=True)
efhk_ilzq.rename(columns={'night_sect': 'night'}, inplace=True)

In [ ]:
def classify_hems_ifr(row):
    if row['night']:
        pass

In [ ]:
def classify_hems(row):
    if row['night']:
        if row['vis_term'] >= 3000 and row['vis_sect'] >= 3000:
            if row['base_term'] >= 1200 and row['base_sect'] >= 1200:
                return 'NIGHT HEMS VFR'
        return 'NIGHT NO VFR'
        
    if row['vis_term'] >= 3000 and row['vis_sect'] >= 3000:
        return 'DAY HEMS VFR'
    if row['vis_term'] >= 2000 and row['vis_sect'] >= 2000:
        if row['ceil_term'] >= 400 and row['ceil_sect'] >= 400:
            return 'DAY HEMS VFR'
    if row['vis_term'] >= 500 and row['vis_sect'] >= 500:
        if row['ceil_term'] >= 500 and row['ceil_sect'] >= 500:
            return 'DAY HEMS VFR'
    return 'DAY NO VFR'

In [ ]:
efhk_ilzq['class'] = efhk_ilzq.apply(classify_hems, axis=1)

In [ ]:
efhk_ilzq

,temp_term,vis_term,rvr_term,ceil_term,base_term,metar_term,temp_sect,vis_sect,ceil_sect,base_sect,night,metar_sect,class
time,,,,,,,,,,,,,
2021-01-01 00:20:00,0,700,2000,200,200,EFHK 010020Z 21002KT 0700 R04L/P2000N R15/1400...,0,1800,9999,400,True,ILZQ 010020Z AUTO 23003KT 1800 BR ////// M00/M...,NIGHT
2021-01-01 00:40:00,0,700,2000,200,200,EFHK 010020Z 21002KT 0700 R04L/P2000N R15/1400...,0,1700,9999,400,True,ILZQ 010040Z AUTO 22003KT 1700 BR ////// M00/M...,NIGHT
2021-01-01 00:50:00,0,1400,2000,200,200,EFHK 010050Z 20002KT 1400 R04L/P2000N R15/P200...,0,1800,9999,400,True,ILZQ 010050Z AUTO 23003KT 1800 BR ////// M00/M...,NIGHT
2021-01-01 01:00:00,0,1400,2000,200,200,EFHK 010050Z 20002KT 1400 R04L/P2000N R15/P200...,0,2500,9999,400,True,ILZQ 010100Z AUTO 22003KT 2500 BR ////// M00/M...,NIGHT
2021-01-01 01:10:00,0,1400,2000,200,200,EFHK 010050Z 20002KT 1400 R04L/P2000N R15/P200...,0,4100,9999,400,True,ILZQ 010110Z AUTO 22002KT 4100 BR ////// M00/M...,NIGHT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-03 09:40:00,-12,9000,9999,9999,400,EFHK 030920Z 36007KT 9000 -SN FEW004 M12/M13 Q...,-13,8000,1800,400,False,ILZQ 030940Z AUTO 36006KT 8000 -SN OVC018 M13/...,DAY HEMS VFR
2021-12-03 09:50:00,-12,10000,9999,9999,400,EFHK 030950Z 36008KT 9999 -SN FEW004 SCT022 M1...,-13,4600,1800,400,False,ILZQ 030950Z AUTO 36004KT 4600 -SN OVC018 M13/...,DAY HEMS VFR
2021-12-03 10:00:00,-12,10000,9999,9999,400,EFHK 030950Z 36008KT 9999 -SN FEW004 SCT022 M1...,-12,5000,1700,400,False,ILZQ 031000Z AUTO 36004KT 5000 -SN OVC017 M12/...,DAY HEMS VFR


In [ ]:
efhk_ilzq.to_csv('output/efhk_ilzq.csv', sep=';')

In [ ]:
efhk_ilzq.to_excel('output/efhk_ilzq.xlsx')

In [ ]:
efhk_ilzq['class'].value_counts()

DAY HEMS VFR    26811
NIGHT           17487
NO VFR            926
Name: class, dtype: int64